In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
from itertools import combinations
from itertools import combinations_with_replacement

In [2]:
# Definimos estados y priors
States = np.array([0,1]) 
Prior = np.array([0.2,0.8])

In [3]:
# Definimos matrices de emisión y transmision
T = np.array([[0.8,0.2],[0.2,0.8]])

E = np.array([[0.5,0.9],[0.5,0.1]])



In [4]:
DictH = {0:'Justa',1:'Sesgada'} 

DictO = {0:'Cara',1:'Sello'}

# Secuencia observada 120 -> VAR
Obs = np.array([1,0,0,0,1,0,1,0])

In [5]:
def GetStates(States,N):
    
    CStates = list( combinations_with_replacement(States,N) )
    
    print(CStates)
    
    Permu = []
    
    for it in CStates:
        p = list(permutations(it,N))
       # print(p)
        
        for i in p:
            if i not in Permu:
                Permu.append(i)
                
    return np.array(Permu)

In [6]:
HiddenStates = GetStates(States,8)
HiddenStates

[(0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 1, 1, 1), (0, 0, 0, 0, 1, 1, 1, 1), (0, 0, 0, 1, 1, 1, 1, 1), (0, 0, 1, 1, 1, 1, 1, 1), (0, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 1)]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]])

In [7]:
#b) Encuentre la secuencia oculta mas probable del tipo de moneda que se eligio en cada lanzamiento y su respectiva probabilidad Pi
def GetProb(T, E, Obs, State, Prior):
    n = len(Obs)
    p = 1.

    p *= Prior[State[0]]

    # Matriz de transición
    for i in range(n - 1):
        p *= T[State[i + 1], State[i]]

    for i in range(n):
        p *= E[Obs[i], State[i]]

    return p


P = np.zeros(HiddenStates.shape[0], dtype=np.float64)
for i in range(P.shape[0]):
    P[i] = GetProb(T,E,Obs,HiddenStates[i],Prior)
ii = np.where( P == np.amax(P))
print(HiddenStates[ii])
print(np.sum(P))

[[1 1 1 1 0 0 0 0]]
0.0019344381513984013


In [8]:
#Calcule las probabilidades de cada estado observable (o) como la suma de las probabilidades de todos los estados ocultos

ObsStates = GetStates([0,1],8)
Nobs = ObsStates.shape[0]

PObs = np.zeros(Nobs)

for j in range(Nobs):
    
    dim = HiddenStates.shape[0]
    P = np.zeros(dim)
    
    for i in range(dim):
        P[i] = GetProb(T,E,ObsStates[j],HiddenStates[i],Prior)
        
    PObs[j] = np.sum(P)

PObs

[(0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 1, 1, 1), (0, 0, 0, 0, 1, 1, 1, 1), (0, 0, 0, 1, 1, 1, 1, 1), (0, 0, 1, 1, 1, 1, 1, 1), (0, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 1)]


array([0.13698241, 0.04114921, 0.0347373 , 0.03175272, 0.03026766,
       0.02932288, 0.02830811, 0.02656789, 0.02297818, 0.01870516,
       0.01330002, 0.01467809, 0.01079433, 0.01067359, 0.01276253,
       0.00956961, 0.00877661, 0.00936674, 0.01172117, 0.00883478,
       0.00776184, 0.00767971, 0.00853533, 0.01088136, 0.00811519,
       0.00697382, 0.00661175, 0.00679511, 0.00764233, 0.00970066,
       0.00694603, 0.00590355, 0.00547305, 0.00537647, 0.00555143,
       0.00611094, 0.0074164 , 0.00955193, 0.00649319, 0.00616157,
       0.00733671, 0.00505945, 0.00428819, 0.00497251, 0.00456283,
       0.0048075 , 0.00623446, 0.00432462, 0.00337544, 0.00382959,
       0.00321043, 0.00330316, 0.00416108, 0.00372302, 0.00355961,
       0.00405611, 0.00553322, 0.00381405, 0.00283376, 0.00316972,
       0.00249084, 0.00251514, 0.00309313, 0.00256378, 0.00241017,
       0.0026765 , 0.00353481, 0.00309969, 0.00278698, 0.0028754 ,
       0.00342205, 0.00478004, 0.00319794, 0.00231328, 0.00256

In [9]:
#d) Verifique que la suma de todos los estados observables es 1.

Verificacion = np.sum(PObs)
Verificacion

1.0000000000000007

In [10]:
#e) Depende el resultado de la probabilidad a-priori?
#Definimos una nueva probabiliad a-priori
Prior2 = np.array([0.4,0.6])

#Bucamos la la secuencia oculta mas probable con la nueva probabiliad a-priori
P2 = np.zeros(HiddenStates.shape[0], dtype=np.float64)
for i in range(P.shape[0]):
    P2[i] = GetProb(T,E,Obs,HiddenStates[i],Prior2)
ii = np.where( P2 == np.amax(P2))
print(HiddenStates[ii])
print(np.sum(P2))



[[0 0 0 0 0 0 0 0]]
0.0025033839429888013


Como se evidencia el resultado si depende de la probabilidad a-priori